In [4]:
params_dict = {
"outputDir"     :   "none",
"eosType"       :   "0",           
"etaS"          :   "0.12",            #  eta/s value
"zetaS"         :   "0.0",             #  zeta/s, bulk viscosity
"e_crit"        :   "0.515",           #  criterion for surface finding
"zetaSparam"    :   "3",               #  0 basic param (is zetaS 0 no bulk), 1,2,3
"nx"            :   "101",             # number of cells in X direction
"ny"            :   "101",             # number of cells in Y direction
"nz"            :   "51",              # number of cells in eta direction
"xmin"          :   "-20.0",           # coordinate of the first cell
"xmax"          :   "20.0",            # coordinate of the last cell
"ymin"          :   "-20.0",
"ymax"          :   "20.0",
"etamin"        :   "-10.0",
"etamax"        :   "10.0",
"icModel"       :   "5",
"glauberVar"    :   "1",       	      #not used
"icInputFile"   :   "none",
"s0ScaleFactor" :   "53.55",	      #not used in glissando (glauber + rapidity)
"epsilon0"      :   "30.0",	      #not used in glissando (glauber)
"impactPar"     :   "7.0",	      #not used in glissando (glauber)
"alpha"         :   "0.0",             #NEVER used
"tau0"          :   "0.6",  	      # starting proper time
"tauMax"        :   "15.0",  	      # proper time to stop hydro
"dtau"          :   "0.1"   	      # timestep
}


glissando_dict = {
	"sNN"      : "5020",
	"eta0"     : "3.7", #/2.3 // midrapidity plateau
  	"sigEta"   : "1.4", # diffuseness of rapidity profile
  	"etaM"     : "4.5",
  	"ybeam"    : "8.585", # beam rapidity
  	"alphaMix" : "0.15", # WN/binary mixing
  	"Rg"	   : "0.4", # Gaussian smearing in transverse dir
  	"A" 	   : "0.0",  # 5e-4; // initial shear flow
  	"neta0"	   : "0.0" 
}

supermc_dict = {
	"sNN"      : "5020",
	"eta0"     : "3.5", # midrapidity plateau
  	"sigmaeta" : "1.4", # diffuseness of rapidity profile
  	"w"	   : "0.4",
  	"eff"	   : "0.15" 
}

input_dict = {
        "param" : "vhlle param file (-param)",
        "system" : "vhlle collision system (-system)",
        "icinput" : "initial condition file in vhlle (-ISinput)",
        "outputfolder" : "master output folder, base of the path three",
        "Nevents" : 100
}

sampler_config = {
        "surface"          : "/path/to/freezeout/surface",
        "spectra_dir"      : " /path/to/output",
        "number_of_events" : 100,
        "weakContribution" : 0,
        "shear"            : 1,
        "ecrit"            : 0.5
}

smash_yaml_config={
        'Version': 1.8,
        'Logging': {'default': 'INFO'},
        'General': {'Modus': 'List',
        'Time_Step_Mode': 'None',
        'Delta_Time': 0.1,
        'End_Time': 30000.0,
        'Randomseed': -1,
        'Nevents': 100},
        'Output': {'Particles': {'Format': ['Binary', 'Oscar2013']}},
        'Modi': {'List': {'File_Directory': '../build/',
        'File_Prefix': 'sampling',
        'Shift_Id': 0}}
}

dict_list = [params_dict,glissando_dict,supermc_dict, input_dict, sampler_config, smash_yaml_config]

In [24]:
def modify_dictionary(dict_name,**kwargs):
	if dict_name == "params_dict":
		dict_copy = params_dict.copy()
	if dict_name == "glissando_dict":
		dict_copy = glissando_dict.copy()
	if dict_name == "supermc_dict":
		dict_copy = supermc_dict.copy()
	if dict_name == "input_dict":
		dict_copy = input_dict.copy()
	if dict_name == "sampler_config":
		dict_copy = input_dict.copy()
	if dict_name == "smash_yaml_config":
		dict_copy = input_dict.copy()

	for key, value in kwargs.items():
		if key in dict_copy.keys():
			dict_copy[key] = value
		else:
			print("ERROR: the key you want to modify is not in this dictionary!")
			return
			
	return dict_copy

In [23]:
dict_modify("supermc_dict",eta0="2")

{'sNN': '5020', 'eta0': '2', 'sigmaeta': '1.4', 'w': '0.4', 'eff': '0.15'}

In [25]:
import hybrid_lib as hl
import subprocess
def get_different_key_value(dictionary1,dictionary2):
    name = "" 
    for key, value in dictionary1.items():
        if key in dictionary2 and value != dictionary2[key]:
            name += key + "_" + str(value) + "_" 
    return name

In [4]:
def name_folder(prefix="", param=hl.params_dict, gliss=hl.glissando_dict, 
                    smc=hl.supermc_dict, sampl=hl.sampler_config, smash=hl.smash_yaml_config):
    name = ""
    if(prefix != ""):
        name += prefix + "_"

    if param != hl.params_dict:
        name += "params:"
        name += get_different_key_value(param,hl.params_dict)  

    if gliss != hl.glissando_dict:
        name += "gliss:" 
        name += get_different_key_value(gliss,hl.glissando_dict)
        
    if smc != hl.supermc_dict:
        name += "superMC:"
        name += get_different_key_value(smc,hl.supermc_dict)

    if sampl != hl.sampler_config:
        name += "sampler:"
        name += get_different_key_value(sampl, hl.sampler_config)
    
    if smash != hl.smash_yaml_config:
        name += "smash:"
        name += get_different_key_value(smash, hl.smash_yaml_config)
    
    name = name[:-1] #remove the last "_"
    
    return name

In [28]:
import hybrid_lib as hl
import subprocess

def name_path_tree(dict_par,dict,cent):
	# function to name the path tree
	path_name = "system"+ dict["sNN"]+"centrality"+cent + "eta0" + dict["eta0"] + "ecrit" +dict_par["e_crit"] + "sigeta" + dict["sigmaeta"] + "w" + dict["w"] + "f" + dict["eff"]
	return path_name


test_values_etaS = ["12"]
test_values_ecrit = ["5"]
test_values_eta0 = ["40","40.5"]
test_values_sigEta = ["11","14"]
zeta_params_values = ["0","3"]

Nevents = 200
centrality="20-30"
icfile  = "/home/palermo/superMC/wouded_lhc5020_20-30%.data"

for EtaS in test_values_etaS:
	for Ecrit in test_values_ecrit:
		for Eta0 in test_values_eta0:
			for SEta in test_values_sigEta:
				for zz in zeta_params_values:
					copy_params = hl.modify_dictionary("params_dict", etaS=EtaS, e_crit=Ecrit,zetaSparam=zz)
					copy_superMC_setup = hl.modify_dictionary("supermc_dict", eta0=Eta0,sigmaeta=SEta)
					name_maindir = hl.name_folder(param=copy_params,smc=copy_superMC_setup,prefix="cent"+centrality)
					print(name_maindir)
				# hl.print_dict_to_file(copy_params,path_params_file)
				# hl.print_dict_to_file(copy_superMC_setup,path_supermc_file)

				# hl.run_hybrid(path_params_file,path_supermc_file,icfile,name_maindir)
				# hl.analysis_and_plots(name_maindir,Nevents)                  
				# subprocess.run(["rm -r ",path_tree["hydro"],path_tree["sampler"],path_tree["after"]])

cent20-30_params:etaS_12_e_crit_5_zetaSparam_0_superMC:eta0_40_sigmaeta_11
cent20-30_params:etaS_12_e_crit_5_superMC:eta0_40_sigmaeta_11
cent20-30_params:etaS_12_e_crit_5_zetaSparam_0_superMC:eta0_40_sigmaeta_14
cent20-30_params:etaS_12_e_crit_5_superMC:eta0_40_sigmaeta_14
cent20-30_params:etaS_12_e_crit_5_zetaSparam_0_superMC:eta0_40.5_sigmaeta_11
cent20-30_params:etaS_12_e_crit_5_superMC:eta0_40.5_sigmaeta_11
cent20-30_params:etaS_12_e_crit_5_zetaSparam_0_superMC:eta0_40.5_sigmaeta_14
cent20-30_params:etaS_12_e_crit_5_superMC:eta0_40.5_sigmaeta_14
